In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType
from pyspark.sql.functions import from_json

# SparkSession başlatmak ve Kafka bağımlılığını yüklemek
spark = SparkSession.builder \
    .appName("RealTimeAnomalyDetection") \
    .config("spark.jars", "C:/Users/Buse Yener/Downloads/spark-sql-kafka-0-10_2.12-3.3.0.jar") \
    .config("spark.sql.streaming.checkpointLocation", "file:///C:/checkpoint_location") \
    .getOrCreate()

# Kafka'ya bağlanacak
spark.sparkContext.setLogLevel("WARN")

# Schema tanımlaması (dataframe'in doğru formatta olması için)
schema = StructType([
    StructField("V1", FloatType()),
    StructField("V6", FloatType()),
    StructField("V8", FloatType()),
    StructField("V13", FloatType()),
    StructField("V15", FloatType()),
    StructField("V19", FloatType()),
    StructField("V20", FloatType()),
    StructField("V21", FloatType()),
    StructField("V23", FloatType()),
    StructField("V24", FloatType()),
    StructField("V25", FloatType()),
    StructField("V26", FloatType()),
    StructField("V27", FloatType()),
    StructField("V28", FloatType()),
    StructField("year", FloatType()),
    StructField("month", FloatType()),
    StructField("day", FloatType()),
    StructField("hour", FloatType()),
    StructField("weekday", FloatType()),
    StructField("log_amount", FloatType()),
    StructField("Class", FloatType())
])

# Kafka'dan veri okuma
input_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "input_topic") \
    .load()

# Kafka verisini çözümle (value'yı STRING formatında almak)
json_data = input_stream.selectExpr("CAST(value AS STRING) as value")

# JSON verisini çözümleme
parsed_data = json_data.select(from_json("value", schema).alias("data"))

# Veriyi düzleştir (data sütunundaki her bir alt sütunu alıyoruz)
final_data = parsed_data.select("data.*")

# Anomali sınıflandırma işlemi veya diğer işlemler burada yapılabilir
# final_data.show()  # Bu komut yalnızca veriyi göstermek için kullanılır

# Streaming işlemi yapmak için output bir yerde depolanabilir (örneğin konsola veya dosyaya yazılabilir)
query = final_data.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# Query'nin durmasını bekleyin (Bu işlem streaming işlemi çalıştırmaya devam eder)
query.awaitTermination()


Py4JJavaError: An error occurred while calling o137.load.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.sql.kafka010.KafkaSourceProvider$
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateStreamOptions(KafkaSourceProvider.scala:338)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.sourceSchema(KafkaSourceProvider.scala:71)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:233)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:118)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:118)
	at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:36)
	at org.apache.spark.sql.streaming.DataStreamReader.loadInternal(DataStreamReader.scala:169)
	at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:145)
	at 